# HRRR Wind Roses from OSG
Brian Blaylock  
October 6, 2017

Histogram bins for series of HRRR wind speed and direction
Shows how to plot data from wind roses binned by the Open Science Grid.

In [1]:
%matplotlib inline
import numpy as np
from datetime import datetime, timedelta
from netCDF4 import Dataset
import multiprocessing # :)
import matplotlib.pyplot as plt
import h5py

import sys
sys.path.append('/uufs/chpc.utah.edu/common/home/u0553130/pyBKB_v2/')
from BB_downloads.HRRR_S3 import get_hrrr_variable
from BB_wx_calcs.wind import wind_uv_to_spd, wind_uv_to_dir

In [2]:
def get_HRRR_value(getthisDATE):
    """
    Getting HRRR data, just return the value (not the latitude and longitude)
    """
    H = get_hrrr_variable(getthisDATE, variable, fxx=fxx, model='hrrr', field='sfc', value_only=True, verbose=False)
    if H['value'] is None:
        print "!! WARNING !! COULD NOT GET", getthisDATE
    return H['value']

In [3]:
def histogram(direction, var, bins, nsector, normed=False, blowto=False):
    """
    From windrose: https://github.com/python-windrose/windrose/
    Returns an array where, for each sector of wind
    (centred on the north), we have the number of time the wind comes with a
    particular var (speed, polluant concentration, ...).
    * direction : 1D array - directions the wind blows from, North centred
    * var : 1D array - values of the variable to compute. Typically the wind
      speeds
    * bins : list - list of var category against we're going to compute the table
    * nsector : integer - number of sectors
    * normed : boolean - The resulting table is normed in percent or not.
    * blowto : boolean - Normaly a windrose is computed with directions
    as wind blows from. If true, the table will be reversed (usefull for
    pollutantrose)
    """
    if len(var) != len(direction):
        raise(ValueError("var and direction must have same length"))

    angle = 360. / nsector

    dir_bins = np.arange(-angle / 2, 360. + angle, angle, dtype=np.float)
    dir_edges = dir_bins.tolist()
    dir_edges.pop(-1)
    dir_edges[0] = dir_edges.pop(-1)
    dir_bins[0] = 0.

    var_bins = bins.tolist()
    var_bins.append(np.inf)

    if blowto:
        direction = direction + 180.
        direction[direction >= 360.] = direction[direction >= 360.] - 360

    table = np.histogram2d(x=var, y=direction, bins=[var_bins, dir_bins], normed=False)[0]
    # add the last value to the first to have the table of North winds
    table[:, 0] = table[:, 0] + table[:, -1]
    # and remove the last col
    table = table[:, :-1]
    if normed:
        table = table * 100 / table.sum()
 
    return dir_edges, var_bins, table

In [4]:
def get_hist_table(ij):
    """
    Function for multiprocessing.
    Create historgram table for each grid
    """
    i, j = ij
    sector_edges, bins, table = histogram(WDIR[:, i, j], WSPD[:, i, j], speed_bins, num_dir_sectors)
    return table

In [5]:
# =============================================================================
# Input arguments (dates should represents the valid time)
month = 9
day = 15
hour = 0
fxx = 0

# Archvie Date Range
sDATE = datetime(2015, 4, 18, hour)
eDATE = datetime(2017, 8, 1, hour)
# =============================================================================

In [6]:
# Range of dates for the desired Month, Hour, and Day 
# (e.g. all 0100 UTC for 15 days before and after the 6th day of the month of January)
# requestDATES is the file we want from the Pando archive, and we will account
# for the offset with the fxx argument.
years = range(sDATE.year, eDATE.year+1)
validDATES = []
for y in years:
    try:
        centerDATE = datetime(y, month, day, hour)
        print datetime(y, month, day, hour), '    fxx:', fxx
    except:
        print 'Not a valid date: datetime(%s, %s, %s, %s)' % (y, month, day, hour)
        continue
    begin = centerDATE-timedelta(days=15)
    end = centerDATE+timedelta(days=15)
    chunk = [begin+timedelta(days=x) for x in range(0,(end-begin).days) if begin+timedelta(days=x) > sDATE]
    validDATES += chunk

requestDATES = np.array([d-timedelta(hours=fxx) for d in validDATES])

2015-09-15 00:00:00     fxx: 0
2016-09-15 00:00:00     fxx: 0
2017-09-15 00:00:00     fxx: 0


In [7]:
# Multiprocessing :) Get field of each item in list requestDATES
cpu_count = multiprocessing.cpu_count()
    
variable = 'UGRD:10 m'
print "getting U grids...",
p = multiprocessing.Pool(cpu_count)
U = p.map(get_HRRR_value, requestDATES)
U = np.array([x for x in U if x is not None])
p.close()
print 'done.'

variable = 'VGRD:10 m'
print "getting V grids...",
p = multiprocessing.Pool(cpu_count)
V = p.map(get_HRRR_value, requestDATES)
V = np.array([x for x in V if x is not None])
p.close()
print "done."

# Convert U and V to Speed and Direction
WSPD = wind_uv_to_spd(U, V)
WDIR = wind_uv_to_dir(U, V)

# We need the HRRR lat/lon arrays, so get a single H dictionary
H = get_hrrr_variable(requestDATES[0], 'UGRD:10 m')

getting U grids... done.
getting V grids... done.
./temp_2015083100_f00_UGR.grib2
Oh, good, you requested a date that should be on Pando.
matched a variable 52:36726332:d=2015083100:UGRD:10 m above ground:anl

range: 36726332 37889002


In [8]:
# Histogram bins
num_dir_sectors = 8               # Number of directional sectors (sectors calculated in histogram function)
speed_bins = np.arange(0, 10, 2)  # Speed bins
num_spd_bins = len(speed_bins)    # Number of speed bins

rose_dim = num_spd_bins * num_dir_sectors
x_dim, y_dim = H['value'].shape


# make rose table for each point in HRRR
indexes = [[i, j] for i in range(x_dim) for j in range(y_dim)]

print "making histogram tables...",
p = multiprocessing.Pool(cpu_count)
result = np.array(p.map(get_hist_table, indexes))
p.close()
print "done!"

rose_storage = result.reshape(x_dim, y_dim, num_spd_bins, num_dir_sectors)

# Get the hisotgram bins to return
sector_edges, bins, table = histogram(WDIR[:, 0, 0], WSPD[:, 0, 0], speed_bins, num_dir_sectors)

timer = datetime.now() - timer1

# Write the HDF5 file of the data
var_name = 'ROSE_10m'
f = h5py.File('OSG_HRRR_%s_m%02d_d%02d_h%02d_f%02d.h5' % (var_name, month, day, hour, fxx), 'w')
h5_per = f.create_dataset('rose tables', data=rose_storage.astype(int), compression="gzip", compression_opts=9)
h5_count = f.create_dataset('wind speed bins', data=bins)
h5_count = f.create_dataset('wind direction edges', data=sector_edges)
h5_cores = f.create_dataset('cores', data=cpu_count)
h5_timer = f.create_dataset('timer', data=str(timer))
h5_begD = f.create_dataset('Beginning Date', data=validDATES[0].strftime('%Y-%m-%d-%H:00'))
h5_endD = f.create_dataset('Ending Date', data=validDATES[-1].strftime('%Y-%m-%d-%H:00'))
f.close()

# Could add percentiles for the wind speed
# Store counts as integers, not floats, to save memory (difference between 23 MB/file to 22 MB/file)

making histogram tables... done!


NameError: name 'timer1' is not defined